In [1]:
import numpy as np
import pandas as pd
import os, os.path
import sqlite3

In [2]:
# 1.) Wczytujemy pliki

data = ['airlines', 'airports', 'flights', 'planes', 'weather']

In [7]:
pd.read_csv('nycflight13/nycflights13_' + 'airlines' + '.csv.gz', 
            compression = 'gzip',
            comment = '#')

,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.
3,B6,JetBlue Airways
4,DL,Delta Air Lines Inc.
5,EV,ExpressJet Airlines Inc.
6,F9,Frontier Airlines Inc.
7,FL,AirTran Airways Corporation
8,HA,Hawaiian Airlines Inc.
9,MQ,Envoy Air


In [13]:
# globals() # słownik przechowuje zmienne
for d in data:
    globals()[d] = pd.read_csv('nycflight13/nycflights13_' + d + '.csv.gz', 
                               compression = 'gzip',
                               comment = '#')

In [16]:
planes.head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan


In [17]:
planes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3322 entries, 0 to 3321
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   tailnum       3322 non-null   object 
 1   year          3252 non-null   float64
 2   type          3322 non-null   object 
 3   manufacturer  3322 non-null   object 
 4   model         3322 non-null   object 
 5   engines       3322 non-null   int64  
 6   seats         3322 non-null   int64  
 7   speed         23 non-null     float64
 8   engine        3322 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 233.7+ KB


In [18]:
# Tworzymy baze danych:
dbname = 'nycflights13.db'

conn = sqlite3.connect(dbname)

In [19]:
for d in data:
    globals()[d].to_sql(d, conn)

In [21]:
# Przykładowa kwerenda:
pd.read_sql_query(""" SELECT * FROM flights LIMIT 10 """, conn)

,index,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
1,1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00
2,2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00
3,3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00
4,4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00
5,5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01 05:00:00
6,6,2013,1,1,555.0,600,-5.0,913.0,854,19.0,B6,507,N516JB,EWR,FLL,158.0,1065,6,0,2013-01-01 06:00:00
7,7,2013,1,1,557.0,600,-3.0,709.0,723,-14.0,EV,5708,N829AS,LGA,IAD,53.0,229,6,0,2013-01-01 06:00:00
8,8,2013,1,1,557.0,600,-3.0,838.0,846,-8.0,B6,79,N593JB,JFK,MCO,140.0,944,6,0,2013-01-01 06:00:00
9,9,2013,1,1,558.0,600,-2.0,753.0,745,8.0,AA,301,N3ALAA,LGA,ORD,138.0,733,6,0,2013-01-01 06:00:00


In [22]:
# Zad. 7.2
# 3

res_sql = pd.read_sql_query("""
SELECT MIN(year) AS minYear, engine,
manufacturer
FROM planes
GROUP BY engine, manufacturer
""", conn)

In [23]:
res_sql

,minYear,engine,manufacturer
0,1975.0,4 Cycle,CESSNA
1,NaN,4 Cycle,JOHN G HESS
2,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,Reciprocating,BARKER JACK L
5,1959.0,Reciprocating,CESSNA
6,2007.0,Reciprocating,CIRRUS DESIGN CORP
7,1959.0,Reciprocating,DEHAVILLAND
8,1956.0,Reciprocating,DOUGLAS
9,2007.0,Reciprocating,FRIEDEMANN JON


In [26]:
# Odtwarzamy pandasem

g = list(planes.groupby(['engine', 'manufacturer']))

In [28]:
g[1]

(('4 Cycle', 'JOHN G HESS'),
     tailnum  year                      type manufacturer model  engines  \
 686  N315AT   NaN  Fixed wing single engine  JOHN G HESS  AT-5        1   
 
      seats  speed   engine  
 686      2    NaN  4 Cycle  )

In [30]:
planes[['year', 'engine', 'manufacturer']]

,year,engine,manufacturer
0,2004.0,Turbo-fan,EMBRAER
1,1998.0,Turbo-fan,AIRBUS INDUSTRIE
2,1999.0,Turbo-fan,AIRBUS INDUSTRIE
3,1999.0,Turbo-fan,AIRBUS INDUSTRIE
4,2002.0,Turbo-fan,EMBRAER
...,...,...,...
3317,2002.0,Turbo-fan,BOEING
3318,1992.0,Turbo-fan,MCDONNELL DOUGLAS AIRCRAFT CO
3319,2002.0,Turbo-fan,BOEING
3320,1992.0,Turbo-jet,MCDONNELL DOUGLAS CORPORATION


In [35]:
res_pandas = planes[['year', 'engine', 'manufacturer']]\
                  .groupby(['engine', 'manufacturer']).min().reset_index()

In [36]:
res_pandas.head()

,engine,manufacturer,year
0,4 Cycle,CESSNA,1975.0
1,4 Cycle,JOHN G HESS,NaN
2,Reciprocating,AMERICAN AIRCRAFT INC,NaN
3,Reciprocating,AVIAT AIRCRAFT INC,2007.0
4,Reciprocating,BARKER JACK L,NaN


In [37]:
res_pandas.columns

Index(['engine', 'manufacturer', 'year'], dtype='object')

In [38]:
res_pandas.rename(columns = {'year': 'minYear'}, inplace = True)

In [40]:
res_pandas.head(2)

,engine,manufacturer,minYear
0,4 Cycle,CESSNA,1975.0
1,4 Cycle,JOHN G HESS,NaN


In [42]:
res_sql.equals(res_pandas)

False

In [55]:
res_sql.columns.tolist()

['minYear', 'engine', 'manufacturer']

In [56]:
res_sql.equals( res_pandas.loc[ :, res_sql.columns.tolist() ] )

True

In [ ]:
conn.close()

In [47]:
# Dygresja - indeksowanie

c = pd.Series(np.round(np.random.uniform(0, 1, 10),  2))

In [49]:
c

0    0.58
1    0.97
2    0.95
3    0.92
4    0.18
5    0.78
6    0.58
7    0.54
8    0.11
9    0.66
dtype: float64

In [51]:
d = c.copy()
i = np.arange(10)
np.random.shuffle(i)
d.index = i

In [52]:
d

9    0.58
3    0.97
5    0.95
1    0.92
8    0.18
2    0.78
6    0.58
0    0.54
7    0.11
4    0.66
dtype: float64

In [53]:
c.loc[0], d.loc[0]

(0.58, 0.54)

In [54]:
c.iloc[0], d.iloc[0]

(0.58, 0.58)